# 03-04: CLI 依赖注入与钩子系统实现可扩展的 CLI 架构模式。

In [ ]:
// ========== 1. 依赖注入容器 ==========
type Constructor<T> = new (...args: any[]) => T;
type ServiceToken<T> = string | symbol | Constructor<T>;
interface ServiceDefinition<T> {
  token: ServiceToken<T>;
  factory: () => T;
  singleton: boolean;
  instance?: T;
}
class DIContainer {
  private services = new Map<any, ServiceDefinition<any>>();

  register<T>(
    token: ServiceToken<T>,
    factory: () => T,
    singleton = true
  ): this {
    this.services.set(token, {
      token,
      factory,
      singleton
    });
    return this;
  }

  registerClass<T>(
    constructor: Constructor<T>,
    singleton = true
  ): this {
    return this.register(
      constructor,
      () => this.resolveClass(constructor),
      singleton
    );
  }

  resolve<T>(token: ServiceToken<T>): T {
    const service = this.services.get(token);
    if (!service) {
      if (typeof token === 'function') {
        return this.resolveClass(token as Constructor<T>);
      }
      throw new Error(`Service not found: ${String(token)}`);
    }

    if (service.singleton) {
      if (!service.instance) {
        service.instance = service.factory();
      }
      return service.instance;
    }

    return service.factory();
  }

  private resolveClass<T>(constructor: Constructor<T>): T {
    // 获取构造函数参数类型
    const paramTypes = Reflect.getMetadata('design:paramtypes', constructor) || [];
    const args = paramTypes.map((type: any) => this.resolve(type));
    return new constructor(...args);
  }

  createScope(): DIContainer {
    const scope = new DIContainer();
    this.services.forEach((def, token) => {
      if (def.singleton) {
        scope.services.set(token, def);
      } else {
        scope.register(token, def.factory, false);
      }
    });
    return scope;
  }
}
// ========== 2. 使用依赖注入 ==========
interface Logger {
  log(message: string): void;
  error(message: string): void;
}
class ConsoleLogger implements Logger {
  log(message: string): void {
    console.log(`[LOG] ${message}`);
  }
  
  error(message: string): void {
    console.error(`[ERROR] ${message}`);
  }
}
interface Config {
  apiUrl: string;
  apiKey: string;
}
class ApiClient {
  constructor(
    private config: Config,
    private logger: Logger
  ) {}

  async fetchData(endpoint: string): Promise<any> {
    this.logger.log(`Fetching ${endpoint}`);
    // 模拟 API 调用
    return { data: 'success' };
  }
}
// 配置容器
const container = new DIContainer();

container.register<Logger>('Logger', () => new ConsoleLogger());

container.register<Config>('Config', () => ({
  apiUrl: 'https://api.example.com',
  apiKey: process.env.API_KEY || ''
}));

container.registerClass(ApiClient);

// 使用
const apiClient = container.resolve(ApiClient);
apiClient.fetchData('/users');

In [ ]:
// ========== 3. 钩子系统 ==========
type HookCallback<T = any> = (context: T) => T | Promise<T>;
type HookPriority = 'high' | 'normal' | 'low';
interface Hook<T = any> {
  name: string;
  callback: HookCallback<T>;
  priority: number;
}
class HookSystem {
  private hooks = new Map<string, Hook[]>();

  register<T>(
    hookName: string,
    callback: HookCallback<T>,
    priority: HookPriority = 'normal'
  ): () => void {
    const priorityMap = { high: 100, normal: 50, low: 10 };
    
    const hook: Hook<T> = {
      name: hookName,
      callback,
      priority: priorityMap[priority]
    };

    if (!this.hooks.has(hookName)) {
      this.hooks.set(hookName, []);
    }

    const hooks = this.hooks.get(hookName)!;
    hooks.push(hook);
    
    // 按优先级排序
    hooks.sort((a, b) => b.priority - a.priority);

    // 返回取消注册函数
    return () => {
      const index = hooks.indexOf(hook);
      if (index > -1) {
        hooks.splice(index, 1);
      }
    };
  }

  async execute<T>(hookName: string, context: T): Promise<T> {
    const hooks = this.hooks.get(hookName) || [];
    let result = context;

    for (const hook of hooks) {
      try {
        result = await hook.callback(result);
      } catch (error) {
        console.error(`Hook ${hookName} failed:`, error);
      }
    }

    return result;
  }

  async executeParallel<T>(hookName: string, context: T): Promise<T[]> {
    const hooks = this.hooks.get(hookName) || [];
    
    const results = await Promise.allSettled(
      hooks.map(hook => hook.callback(context))
    );

    return results
      .filter((r): r is PromiseFulfilledResult<T> => r.status === 'fulfilled')
      .map(r => r.value);
  }

  hasHook(hookName: string): boolean {
    const hooks = this.hooks.get(hookName);
    return !!hooks && hooks.length > 0;
  }
}
// ========== 4. CLI 中使用钩子 ==========
interface CommandContext {
  command: string;
  args: string[];
  options: Record<string, any>;
  exitCode: number;
  errors: Error[];
}
class CLIWithHooks {
  private hooks = new HookSystem();

  constructor() {
    // 注册默认钩子
    this.registerDefaultHooks();
  }

  private registerDefaultHooks(): void {
    // 初始化前钩子
    this.hooks.register<CommandContext>('beforeInit', async (ctx) => {
      console.log('Initializing...');
      return ctx;
    });

    // 执行前钩子
    this.hooks.register<CommandContext>('beforeExecute', async (ctx) => {
      console.log(`Executing command: ${ctx.command}`);
      return ctx;
    });

    // 执行后钩子
    this.hooks.register<CommandContext>('afterExecute', async (ctx) => {
      console.log(`Command completed with exit code: ${ctx.exitCode}`);
      return ctx;
    });
  }

  on<T>(hookName: string, callback: HookCallback<T>, priority?: HookPriority): () => void {
    return this.hooks.register(hookName, callback, priority);
  }

  async run(command: string, args: string[], options: Record<string, any>): Promise<number> {
    let context: CommandContext = {
      command,
      args,
      options,
      exitCode: 0,
      errors: []
    };

    try {
      // 初始化前
      context = await this.hooks.execute('beforeInit', context);

      // 执行前
      context = await this.hooks.execute('beforeExecute', context);

      // 执行命令
      context = await this.executeCommand(context);

      // 执行后
      context = await this.hooks.execute('afterExecute', context);

    } catch (error) {
      context.errors.push(error as Error);
      context.exitCode = 1;
      
      // 错误钩子
      if (this.hooks.hasHook('onError')) {
        context = await this.hooks.execute('onError', context);
      }
    }

    return context.exitCode;
  }

  private async executeCommand(ctx: CommandContext): Promise<CommandContext> {
    // 实际命令执行逻辑
    console.log(`Running ${ctx.command} with args:`, ctx.args);
    
    // 命令特定钩子
    const hookName = `command:${ctx.command}`;
    if (this.hooks.hasHook(hookName)) {
      return this.hooks.execute(hookName, ctx);
    }

    return ctx;
  }
}
// 使用
const cli = new CLIWithHooks();

// 添加自定义钩子
cli.on<CommandContext>('beforeExecute', async (ctx) => {
  console.log('Custom before execute logic');
  return ctx;
}, 'high');

cli.on<CommandContext>('command:deploy', async (ctx) => {
  console.log('Deploying...');
  ctx.exitCode = 0;
  return ctx;
});

// 运行
// cli.run('deploy', ['production'], { verbose: true });